In [ ]:
# ============================
# 🧩 Imports
# ============================
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import librosa
from tqdm import tqdm
from tkinter import Tk, filedialog

# ============================
# ⚙️ Configuration
# ============================
DEFAULT_SR = 16000  # Default sampling rate (you can change this)
WINDOW_SEC = 1.0
HOP_SEC = 0.5

# ============================
# 🧠 Load pretrained models
# ============================
print("🔹 Loading YAMNet model...")
yamnet_model = hub.load("https://tfhub.dev/google/yamnet/1")

print("🔹 Loading drone classifier model...")
drone_model = tf.keras.models.load_model("yamnet_drone_classifier.h5")  # make sure this file is in the same folder

# ============================
# 🎧 Load full audio file
# ============================
def load_full_audio(file_path, sr=DEFAULT_SR):
    """
    Load an audio file and resample to the given sampling rate.
    """
    audio, _ = librosa.load(file_path, sr=sr, mono=True)
    return audio

# ============================
# 🪟 Split audio into windows
# ============================
def sliding_window(audio, sr=DEFAULT_SR, window_sec=WINDOW_SEC, hop_sec=HOP_SEC):
    """
    Split audio into overlapping windows.
    """
    window_size = int(window_sec * sr)
    hop_size = int(hop_sec * sr)
    windows = []
    for start in range(0, len(audio) - window_size + 1, hop_size):
        segment = audio[start:start + window_size]
        windows.append(segment)
    # Handle short files
    if not windows:
        pad = np.pad(audio, (0, max(0, window_size - len(audio))))
        windows.append(pad)
    return np.array(windows)

# ============================
# 🔍 Analyze full audio
# ============================
def analyze_audio(file_path, sr=DEFAULT_SR, window_sec=WINDOW_SEC, hop_sec=HOP_SEC):
    """
    Analyze an audio file and detect drone presence over time.
    """
    audio = load_full_audio(file_path, sr=sr)
    windows = sliding_window(audio, sr=sr, window_sec=window_sec, hop_sec=hop_sec)
    times = np.arange(len(windows)) * hop_sec

    preds = []
    print(f"\n🔍 Analyzing {len(windows)} audio segments...")
    for segment in tqdm(windows, desc="Processing"):
        segment_tensor = tf.convert_to_tensor(segment, dtype=tf.float32)
        # Extract embeddings using YAMNet
        scores, embeddings, spectrogram = yamnet_model(segment_tensor)
        mean_embedding = tf.reduce_mean(embeddings, axis=0, keepdims=True)
        # Predict with trained model
        pred = drone_model.predict(mean_embedding, verbose=0)
        preds.append(pred[0])

    preds = np.array(preds)
    return times, preds

# ============================
# 📊 Plot detection results
# ============================
def plot_detection(times, preds, threshold=0.5):
    plt.figure(figsize=(12, 4))
    plt.plot(times, preds[:, 0], label="Drone Probability", color="red")
    plt.axhline(threshold, color='gray', linestyle='--', label='Threshold')
    plt.xlabel("Time (s)")
    plt.ylabel("Probability")
    plt.title("Drone Detection Timeline")
    plt.legend()
    plt.grid(True)
    plt.show()

# ============================
# 🧪 Main Execution
# ============================
if __name__ == "__main__":
    # Choose audio file interactively
    root = Tk()
    root.withdraw()  # Hide tkinter window
    file_path = filedialog.askopenfilename(
        title="Select an audio file",
        filetypes=[("Audio Files", "*.wav *.mp3 *.flac *.ogg")]
    )
    
    if not file_path:
        print("❌ No file selected.")
        exit()

    print(f"\n✅ Selected file: {file_path}")

    # Choose sampling rate
    try:
        sr = int(input(f"Enter sampling rate (default = {DEFAULT_SR}): ") or DEFAULT_SR)
    except ValueError:
        sr = DEFAULT_SR

    print(f"🎚 Using sampling rate: {sr} Hz")

    # Run analysis
    times, preds = analyze_audio(file_path, sr=sr)
    plot_detection(times, preds)


🔹 Loading YAMNet model...


🔹 Loading drone classifier model...
